# Exercise 8 - R: Anomaly Detection and Recommender Systems

<img width="350px" src="https://upload.wikimedia.org/wikipedia/commons/0/07/Two-dimensional_Outliers_Example.png">

## *Part I*: Anomaly Detection

In this section, we will determine outliers in a dataset by using a statistical approach.  Specifically, we will assume that our variables have a [Gaussian (or normal) distribution](https://en.wikipedia.org/wiki/Normal_distribution), calculate the [probability density function (PDF)](https://en.wikipedia.org/wiki/Probability_density_function) to see the relative likelihood of  that point appearing (the actual definition is a lot more nuanced since the probability of any actual point will always be 0, but that's statistics for you), and then select the cutoff value that maximizes the [F1 score](https://en.wikipedia.org/wiki/F1_score) using a validation dataset.  We will then mark any point with a probability less than our cutoff value as an outlier.

This is technically a supervised approach since we're using labeled data and a validation set, but there are a plethora of unsupervised techniques as well.  Here are a few others for example:

- **Distance/Density based approaches:** [Local outlier factor](https://en.wikipedia.org/wiki/Local_outlier_factor), [clustering](https://en.wikipedia.org/wiki/Cluster_analysis), or [k-means](https://en.wikipedia.org/wiki/K-means_clustering)

- **Statistical approaches:** [Tukey's range test](https://en.wikipedia.org/wiki/Tukey%27s_range_test), [generalized extreme Studentized deviate (GESD)](http://www.itl.nist.gov/div898/handbook/eda/section3/eda35h3.htm), and others

We're also using the words "anomaly" and "outlier" interchangeably, but there is often debate between the terms.  Generally speaking, an outlier is a value that is extremely unlikely (an improbability), and an anomaly is an invalid value (or impossibility) that was generated by a different process altogether (which goes back into statistical theory and the importance of the data generation process).  Regardless, we're going to continue using these two terms as synonyms because we are only concerned with detecting them rather than debating their validity.

---

Beginning with package imports and data loading

In [1]:
rm(list=ls())  # Clearing all environment variables

suppressPackageStartupMessages({
  library(R.matlab)  # Reads MATLAB/Octave matrices
  library(repr)  # Resizing plot sizes in Jupyter
  library(tidyverse)  # dplyr/tidyr/ggplot
})

theme_update(plot.title = element_text(hjust = 0.5))  # Centers ggplot2 titles
options(repr.plot.width=7, repr.plot.height=4.5)  # Adjusts plot sizes

Warning message:
"package 'R.matlab' was built under R version 3.3.3"Warning message:
"package 'repr' was built under R version 3.3.3"Warning message:
"package 'tidyverse' was built under R version 3.3.3"Warning message:
"package 'tibble' was built under R version 3.3.3"Warning message:
"package 'tidyr' was built under R version 3.3.3"Warning message:
"package 'readr' was built under R version 3.3.3"Warning message:
"package 'purrr' was built under R version 3.3.3"Warning message:
"package 'dplyr' was built under R version 3.3.3"

In [2]:
# Reading in data
data <- readMat('ex8/ex8data1.mat')
X <- data$X
Xval <- data$Xval
yval <- data$yval
rm(data)  # Clears from environment

# Creating a data frame for plotting
df <- data.frame(X)
colnames(df) <- c('Latency_ms', 'Throughput_mbs')

# Plotting the data
ggplot(df) +
  geom_point(aes(x=Latency_ms, y=Throughput_mbs), alpha=0.4)

### Gaussian Distribution

Before performing anomaly detecting, we need to fit a model to the data's distribution.

Estimate the Gaussian distribution for each of the features $x_i$.  For each feature $i=1 \dots n$, find parameters $\mu_i$ and $\sigma_i^2$ that fit the data in the $i$-th dimension $\{x_i^{(1)}, \dots, x_i^{(m)}\}$ (the $i$-th dimension of each example)

The Gaussian distribution is given by
$$p(x; \mu, \sigma^2) = \frac{1}{\sqrt{2 \pi \sigma^2}} e ^ {-\frac{(x - \mu)^2}{2 \sigma^2}}
\\
\mu: \text{Mean}
\\
\sigma^2: \text{Controls the variance}$$

### Estimating Gaussian parameters

In order to estimate the parameters $(\mu_i, \sigma_i^2)$, of the $i$-th feature, we will use the following equations for the mean and variance:

$$\text{Mean: } \mu_i = \frac{1}{m} \sum_{j=1}^m x_i^{(j)}
\\
\text{Variance: } \sigma_i^2 = \frac{1}{m} \sum_{j=1}^m (x_i^{(j)} - \mu_i)^2$$

In [3]:
estimate_gaussian <- function(X) {
  # Takes the matrix X as an input and outputs:
  # mu: n-dimensional vector that holds the mean of all the n features
  # sigma2: n-dimensional vector that holds the variance of all features
  
  mu <- colMeans(X)
  sigma2 <- apply(X, 2, var)
  return(list('mu' = mu, 'sigma2' = sigma2))
}


estimate_gaussian(X)

$mu
[1] 14.11223 14.99771

$sigma2
[1] 1.838620 1.715333

Next, we'll plot the contour plot using our obtained mean and variance values

I borrowed some code from [kaleko's notebook](https://github.com/kaleko/CourseraML/blob/master/ex8/ex8.ipynb) in order to generate the contour plot (if you're reading this, thank you!)

### Selecting the threshold $\epsilon$

With the Gaussian parameters, we can see which points are more or less likely to be outliers according to their probabilities in the distribution.  We can determine the threshold for outliers with the cross validation set.

We'll do this by optimizing the $F_1$ score on the cross validation set.  As a reminder, the $F_1$ score is a mix of precision and recall:

$$F_1 = \frac{2 \cdot precision \cdot recall}{precision + recall}$$

and precision and recall are calculated as follows:

$$precision = \frac{True Positive}{True Positive + False Positive}
\\
recall = \frac{True Positive}{True Positive + False Negative}$$

In [4]:
select_threshold <- function(X, Xval, yval) {
  # Selects the threshold for determining outliers according to which
  # value minimizes the F1 score on the validatino set
  # Returns the threshold and the F1 score
  
  # Creating variables to be replaced
  optimalEpsilon <- 0
  optimalF1 <- 0
  
  # Generating the probability of the point according to the distribution
  params <- estimate_gaussian(X)
  mu <- params$mu
  sigma2 <- params$sigma2
  # probability <- matrix(0, dim(Xval)[1], dim(Xval)[2])
  probability <- dnorm(Xval, mu, sigma2)
  
  # Trying different cutoff values and evaluating the f1 score
  for (epsilon in seq(min(probability), max(probability),
                      length.out = 1000)) {
    prediction <- probability < epsilon
    
    # Calculating true/false positives/negatives
    truePositive <- sum((yval[, ] == 1) & (prediction == 1))
    falsePositive <- sum((yval[, ] == 0) & (prediction == 1))
    falseNegative <- sum((yval[, ] == 1) & (prediction == 0))
    
    # Calculating the F1 score
    precision <- truePositive / (truePositive + falsePositive)
    recall <- truePositive / (truePositive + falseNegative)
    f1 <- (2 * precision * recall) / (precision + recall)
    
    # Replacing optimal F1/epsilon if a higher value was found
    if (!is.na(optimalF1) && !is.na(f1) && optimalF1 < f1) {
      optimalEpsilon <- epsilon
      optimalF1 <- f1
    }
  }
  return(list("f1" = optimalF1, "epsilon" = optimalEpsilon))
}


results <- select_threshold(X, Xval, yval)
results

$f1
[1] 0.7142857

$epsilon
[1] 0.009777896

Now that we have our cutoff, let's assign it to our data set and plot the results

In [5]:
# Assigning Gaussian estimates to variables
mu <- estimate_gaussian(X)$mu
sigma2 <- estimate_gaussian(X)$sigma

# Calculating the probability density for the original points
df$LatencyProb <- pnorm(df$Latency_ms, mu[1], sigma2[1])
df$ThroughputProb <- pnorm(df$Throughput_mbs, mu[2], sigma2[2])

# Assigning outliers based on our obtained epsilon
df$Outlier <- ifelse((df$LatencyProb < results$epsilon) | (df$ThroughputProb < results$epsilon), 1, 0)
df$Outlier <- as.factor(df$Outlier)

# Plotting the data with outliers
ggplot(df) +
  geom_point(aes(x=Latency_ms, y=Throughput_mbs, color=Outlier), alpha=0.4)

**[Comment on outliers]**

### High dimensional dataset

This last section of part I does the same things on a dataset that is much more similar to a realistic scenario.  We won't be able to plot this without using a dimensionality reduction algorithm like [$t$-SNE](https://distill.pub/2016/misread-tsne/), but we can at least view our acquired $\epsilon$ and F1 score

In [6]:
# Reading in data
data <- readMat('ex8/ex8data2.mat')
X <- data$X
Xval <- data$Xval
yval <- data$yval
rm(data)  # Clears from environment

# Calculating the parameters of the variables
params <- estimate_gaussian(X)
mu <- params$mu
sigma2 <- params$sigma2
print(params)

# Calculating the cutoff and F1 score
results <- select_threshold(X, Xval, yval)
results

$mu
 [1]   4.939400  -9.637268  13.814707 -10.464489  -7.956229  10.199504
 [7]  -6.019408   7.969829  -6.253182   2.324513   8.473723

$sigma2
 [1] 61.03593 53.25898 58.57404 84.28833 65.33393 89.66454 55.68919 87.24959
 [9] 29.65893 70.85606 50.55412



$f1
[1] 0.1855956

$epsilon
[1] 0.006476572

---

## *Part II*: Recommender Systems

In this section, we'll use [collaborative filtering](https://en.wikipedia.org/wiki/Collaborative_filtering) to generate movie recommendations for users.  More specifically, for each user, we will predict ratings for unrated movies based on how similar users (who gave similar ratings to the same movies as the user in question) rated these movies.  That sentence is a little dense, so a more intuitive way of explaining it is using Amazon's language of "users who liked movie A also liked movie B".

### Moving ratings dataset

I attended the University of Minnesota for grad school, and they are responsible for curating an extremely popular dataset for recommender systems: [MovieLens](https://en.wikipedia.org/wiki/MovieLens).  *Every single professor* talked about it, and I had to sit through countless presentations of student groups using it for projects.  I thought I was finally done with movie ratings and recommender systems, but here we go again.

We have a few things going on with the data we're reading in here:

- **X:** A matrix of features for the movies (ex. genre, actors, directors, etc.).  Each row is a movie, and each column is a feature.  

- **Y:** A matrix of the ratings users provided for movies.  Each row is a user, each column is a movie, and each value is the rating.

- **R:** A matrix of binary flags for if a user provided a rating for a given movie.  This goes along with the Y matrix since we aren't using null values.  Additionally, we want to predict the ratings that users will give the movies they haven't yet rated.

- **$\Theta$:** A matrix of weights provided for the features in X.



$n_u$ = 943 users

$n_m$ = 1682 movies

X = $n_m \times 100$ matrix

$\Theta$ = $n_u \times 100$ matrix

In [7]:
# Reading in the ratings
data <- readMat('ex8/ex8_movies.mat')
R <- data$R
y <- data$Y
rm(data)  # Clears from environment

# Reading in the features
data <- readMat('ex8/ex8_movieParams.mat')
theta <- data$Theta
X <- data$X
rm(data)  # Clears from environment

cat('R & y:', dim(R), 
    '\nX & Theta:', dim(X))

R & y: 1682 943 
X & Theta: 1682 10

The Octave code provided for this exercise plots the y variable to view the distribution of ratings:

In [8]:
image(y, xlab='Movie', ylab='User')
title('Movie Rating by User')

### Collaborative Filtering

[Collaborative Filtering](https://en.wikipedia.org/wiki/Collaborative_filtering) is a way of using "wisdom of the crowd" to recommend items.  More specifically, it recommends items to individuals by finding highly rated items from other users who rated other items similarly to that individual.

Before we can learn the parameter vectors that produce the best fit (minimize the squared error), we need to compute the cost function and the gradient for collaborative filtering.  

Additionally, we're running into the same issue as exercise 4 where SciPy's optimize function doesn't accept multidimensional parameters, so we have to provide our parameters as a vector to our functions.  However, this has to be a *single* vector, so we have to provide our multiple parameters as one into our cost and gradient function.

In order to assist with this, we'll create a few helper functions to flatten and reshape our parameters.

In [9]:
flatten_params <- function(X, theta) {
  # Prepares parameters for input into cost and gradient functions
  # flattenedParams <- c(as.vector(X), as.vector(theta))
  flattendParams <- c(reshape::melt(X)$value, reshape::melt(theta)$value)
  return(flattendParams)
}


reconstruct_params <- function(parameters, X, theta) {
  # Reconstructs parameters into original shape for use within
  # cost and gradient functions
  reconstructedX <- matrix(parameters[1:length(X)],
                           ncol=dim(X)[2])
  
  reconstructedTheta <- matrix(parameters[(length(X)+1):(length(X)+length(theta))],
                               ncol=dim(theta)[2])
  
  return(list(newX = reconstructedX, newTheta = reconstructedTheta))
}

#### Collaborative Filtering Cost Function

$$J(x^{(1)}, ..., x^{(n_m)}, \theta^{(1)}, ..., \theta^{(n_u)}) = \frac{1}{2} \sum_{(i, j):r(i, j) = 1} (\theta^{(j)})^T x^{(i)} -y^{(i, j)})^2$$

As a reminder, the subtext of the summation symbol is telling us to only calculate the cost if the user has rated the movie.

According to the exercise, our cost should be around 22.22.

In [10]:
cost_function <- function(parameters, X, y, R) {
  # Calculates the unregularized cost function for collaborative filtering
  
  # Restructuring into the original shapes
  theta <- reconstruct_params(parameters, X, theta)$newTheta
  X <- reconstruct_params(parameters, X, theta)$newX
  
  # Calculating the cost
  error <- ((X %*% t(theta)) * R) - y
  # error <- X %*% t(theta)
  squaredError <- error^2
  cost <- (1/2) * sum(squaredError)
  
  return(cost)
}


# Reducing the data for testing this function per exercise instructions
# This is to reduce the overall computations needed
num_users <- 4
num_movies <- 5
num_features <- 3

X <- X[1:(num_movies), 1:(num_features)]
theta <- theta[1:(num_users), 1:(num_features )]
y <- y[1:(num_movies), 1:(num_users)]
R <- R[1:(num_movies), 1:(num_users)]

cost_function(flatten_params(X, theta), X, y, R)

[1] 22.2246

#### Collaborative Filtering Gradient

$$\frac{\partial J}{\partial x_k^{(i)}} = \sum_{j:r(i, j) = 1} ((\theta^{(j)})^T x^{(i)} - y^{(i, j)})\theta_k^{(j)}
\\
\frac{\partial J}{\partial \theta_k^{(j)}} = \sum_{i:r(i, j) = 1} ((\theta^{(j)})^T x^{(i)} - y^{(i, j)})x_k^{(i)}$$

In [11]:
gradient <- function(parameters, X, y, R) {
  # Calculates the unregularized gradient for collaborative filtering
  
  # Restructuring into the original shapes
  theta <- reconstruct_params(parameters, X, theta)$newTheta
  X <- reconstruct_params(parameters, X, theta)$newX
  
  # Calculating the error for the gradients
  error <- ((X %*% t(theta)) * R) - y
  
  # Calculating the gradients
  xGradient <- error %*% theta
  thetaGradient <- t(error) %*% X
  
  return(list(xGradient = xGradient, thetaGradient = thetaGradient))
}


gradient(flatten_params(X, theta), X, y, R)

$xGradient
           [,1]     [,2]       [,3]
[1,] -2.5289916 7.575703 -1.8997903
[2,] -0.5681960 3.352650 -0.5233985
[3,] -0.8324071 4.911633 -0.7667788
[4,] -0.3835828 2.263337 -0.3533405
[5,] -0.8037801 4.742718 -0.7404087

$thetaGradient
          [,1]     [,2]      [,3]
[1,] -10.56802 4.627760 -7.160044
[2,]  -3.05099 1.164414 -3.474108
[3,]   0.00000 0.000000  0.000000
[4,]   0.00000 0.000000  0.000000

#### Regularized Cost Function

Now that we have our unregularized cost function and gradient, let's add the regularization terms:

$$J(x^{(1)}, ..., x^{(n_m)}, \theta^{(1)}, ..., \theta^{(n_u)}) = \frac{1}{2} \sum_{(i, j):r(i, j) = 1} (\theta^{(j)})^T x^{(i)} -y^{(i, j)})^2 +
\Big( \frac{\lambda}{2} \sum_{j=1}^{n_u} \sum_{k=1}^n (\theta_k^{(j)})^2 \Big) +
\Big( \frac{\lambda}{2} \sum_{i=1}^{n_m} \sum_{k=1}^n (x_k^{(i)})^2 \Big)$$

According to the exercise, our cost should be around 31.34.

In [12]:
cost_function <- function(parameters, X, y, R, C=0) {
  # Calculates the regularized cost function for collaborative filtering
  
  # Restructuring into the original shapes
  theta <- reconstruct_params(parameters, X, theta)$newTheta
  X <- reconstruct_params(parameters, X, theta)$newX
  
  # Calculating the cost
  error <- ((X %*% t(theta)) * R) - y
  # error <- X %*% t(theta)
  squaredError <- error^2
  cost <- (1/2) * sum(squaredError)
  
  # Calculating the regularization terms
  regTheta <- (C/2) * sum(theta**2)
  regX <- (C/2) * sum(X**2)
  
  return(cost + regX + regTheta)
}


cost_function(flatten_params(X, theta), X, y, R, 1.5)

[1] 31.34406

#### Regularized Gradient

$$\frac{\partial J}{\partial x_k^{(i)}} = \sum_{j:r(i, j) = 1} ((\theta^{(j)})^T x^{(i)} - y^{(i, j)})\theta_k^{(j)} + \lambda x_k^{(i)}
\\
\frac{\partial J}{\partial \theta_k^{(j)}} = \sum_{i:r(i, j) = 1} ((\theta^{(j)})^T x^{(i)} - y^{(i, j)})x_k^{(i)} + \lambda \theta_k^{(j)}$$

In [13]:
gradient <- function(parameters, X, y, R, C=0) {
  # Calculates the regularized gradient for collaborative filtering
  
  # Restructuring into the original shapes
  theta <- reconstruct_params(parameters, X, theta)$newTheta
  X <- reconstruct_params(parameters, X, theta)$newX
  
  # Calculating the error for the gradients
  error <- ((X %*% t(theta)) * R) - y
  
  # Calculating the gradients and regularization terms
  xGradient <- error %*% theta + (C * X)
  thetaGradient <- t(error) %*% X + (C * theta)
  
  # Flattening for optimization function
  gradient <- flatten_params(xGradient, thetaGradient)
  
  return(gradient)
}


gradient(flatten_params(X, theta), X, y, R, 1.5)

[1]  -0.9559634   0.6030809   0.1298562   0.2968440   0.6025268   6.9753551
 [7]   2.7742114   4.0898522   1.0630093   4.9018533  -0.1086111   0.2583982
[13]  -0.8924733   0.6673814  -0.1974793 -10.1398548  -2.2934702  -0.6478748
[19]   1.0928976   2.1013626   0.4824410  -0.7182067  -0.4078409  -6.7656363
[25]  -2.9979142   1.2700667   0.4902654

### Learning Movie Recommendations

Now that we built our cost function and gradient, we'll add a user with their own movie preferences, learn the parameters (rather than using the already provided ones), and then predict movie ratings for our new user.

We'll begin by adding our new user along with their movie preferences.

In [14]:
# Re-loading the full dataset
data <- readMat('ex8/ex8_movies.mat')
R <- data$R
y <- data$Y
data <- readMat('ex8/ex8_movieParams.mat')
theta <- data$Theta
X <- data$X
rm(data)  # Clears from environment

# Initializing "personal" ratings provided by the exercise
# (y, 1) allows easier appending to y
custom_ratings <- matrix(0, dim(y)[1], 1)

# Movie IDs listed in movie_ids.txt
custom_ratings[1] <- 4  # Toy story
custom_ratings[98] <- 2  # Silence of the Lambs
custom_ratings[7] <- 3
custom_ratings[12] <- 5
custom_ratings[54] <- 4
custom_ratings[64] <- 5
custom_ratings[66] <- 3
custom_ratings[69] <- 5
custom_ratings[183] <- 4
custom_ratings[226] <- 5
custom_ratings[355] <- 5

Next, we'll add this new user to our data set and train the model.

One thing to note here is that we are going to "normalize" our ratings ($y$) by subtracting each user's mean rating from all of their movies that contain ratings.  This will help account for how users rate movies differently by finding their baseline and only looking at how ratings deviate from it.

In [15]:
# Adding new ratings to the data set
custom_R <- ifelse(custom_ratings == 0, 0, 1)
y <- cbind(y, custom_ratings)
R <- cbind(R, custom_R)

# Adjusting theta for the new ratings
theta <- rbind(theta, sample(1:10, 10))

# Setting the initial parameters
initialParams <- runif(length(flatten_params(X, theta)), 0, 1)

# "Normalizing" Y by removing the mean of each row to account for users who rated nothing
yMean <- colSums(y) / colSums(R)

yNorm <- y - yMean
yNorm <- ifelse(yNorm < 0, 0, yNorm)  # Replacing negatives with 0s

# Finding the optimal values for X and Theta
optimalParams <- optim(
      # Specifying function parameters
      par=initialParams,  # Initial guess
      fn=cost_function,  # Function to minimize
      gr=gradient,
      X=X,
      y=yNorm,
      R=R,
      C=10,
      method="CG",  #  Optimization function to use
      control = list(maxit = 200)  # Maximum number of iterations
     # )$par  # Specifying that we only want the obtained thetas
)

cat('Optimized cost function:', optimalParams$value)

Optimized cost function: 18095.64

In [16]:
optimalX <- reconstruct_params(optimalParams$par, X, theta)$newX
optimalTheta <- reconstruct_params(optimalParams$par, X, theta)$newTheta

cat('Optimal X:\n', optimalX[1:30], '\n\nOptimal Theta:\n', optimalTheta[1:30])

Optimal X:
 0.2540339 0.3512182 0.1415992 0.2748039 0.03814674 0.1147413 0.2089475 0.5344474 -0.1755805 0.3281557 0.1773876 0.5153602 0.09519626 0.4037849 0.6623481 0.0816961 0.0009919255 0.02403627 0.170479 0.2189877 0.2572854 0.03039543 0.04851465 0.1124347 0.3981549 0.1012561 0.06746008 0.35454 0.1391227 0.1986501 

Optimal Theta:
 0.4500085 0.0910451 0.1553376 0.2564651 0.475976 0.05692361 0.4743623 0.05761166 0.2690328 -0.03748561 0.2128133 0.1936954 -0.2927539 0.3946172 0.2484188 0.01635861 0.1176759 -0.03310247 0.05437009 -0.04248525 -0.1720551 0.2174895 0.1831599 0.297025 0.3123387 -0.05306918 -0.001541283 -0.1403285 0.154893 0.07632305

Now that we have trained our model, let's look at the top 10 predictions for our user!

In [17]:
# Calculating predictions for all users
all_predictions <- optimalTheta %*% t(optimalX)
all_predictions <- all_predictions + yMean  # de-normalizing

# Selecting predictions for custom user
custom_prediction <- tail(all_predictions, n=1)

# Sorting predictions from highest to lowest
sorted_prediction_idx <- sort(custom_prediction, decreasing = TRUE,index.return=TRUE)$ix

# Printing out the movie scores
# Note: The list of movies aren't loaded in, so we just get the raw scores
for (i in 1:10) {
  ix <- sorted_prediction_idx[i]
  print(custom_prediction[ix])
}

[1] 4.686162
[1] 4.627827
[1] 4.615533
[1] 4.575454
[1] 4.57545
[1] 4.563851
[1] 4.550843
[1] 4.550289
[1] 4.535527
[1] 4.521518


And that's it!  If you followed along, I hope you enjoyed the ride.  I plan on doing the same thing with a deep learning MOOC, and I'll put the exercises in the same folder if you're interested.

I plan on writing a blog post to my [github pages](https://jeffmacaluso.github.io/) about this experience after finishing the rest of the exercises in R and Julia.  I'll update this section with a link to it after I do so, but feel free to raise an issue if you see it on my github pages and I haven't added the link to it here. 